# Remote Execution with ras-commander

This notebook demonstrates how to execute HEC-RAS plans using:
1. **Local parallel execution** - `RasCmdr.compute_parallel()` on your local machine
2. **Remote execution** - `compute_parallel_remote()` on remote machines via PsExec/Docker

## Features
- Distributed execution across multiple remote machines
- Automatic project deployment via network shares
- Parallel execution with configurable workers
- Result collection and consolidation
- **Automatic PsExec.exe download** (no manual setup required)

## Requirements for Remote Execution
- Remote machine(s) configured per `REMOTE_WORKER_SETUP_GUIDE.md`
- Network share accessible from control machine
- HEC-RAS installed on remote machine(s)

---
# Part 1: Setup and Imports

In [1]:
# Standard imports
import sys
import os
import time
from pathlib import Path

# For Development Mode: add parent directory to path
current_file = Path(os.getcwd()).resolve()
rascmdr_directory = current_file.parent
if str(rascmdr_directory) not in sys.path:
    sys.path.insert(0, str(rascmdr_directory))

print("Loading ras-commander...")
from ras_commander import *
print(f"Loaded from: {rascmdr_directory}")

Loading ras-commander...
Loaded from: C:\GH\ras-commander


In [2]:
# Extract example project for testing
project_path = RasExamples.extract_project("BaldEagleCrkMulti2D", output_path="example_projects_23_remote_execution_psexec")
print(f"Project extracted to: {project_path}")

# Initialize project
ras = init_ras_project(project_path, "6.6")
print(f"\nProject: {ras.project_name}")
print(f"Available plans: {list(ras.plan_df['plan_number'])}")

2025-12-10 21:10:18 - ras_commander.RasExamples - INFO - Found zip file: c:\GH\ras-commander\examples\Example_Projects_6_6.zip
2025-12-10 21:10:18 - ras_commander.RasExamples - INFO - Loading project data from CSV...
2025-12-10 21:10:18 - ras_commander.RasExamples - INFO - Loaded 68 projects from CSV.
2025-12-10 21:10:18 - ras_commander.RasExamples - INFO - ----- RasExamples Extracting Project -----
2025-12-10 21:10:18 - ras_commander.RasExamples - INFO - Extracting project 'BaldEagleCrkMulti2D'
2025-12-10 21:10:18 - ras_commander.RasExamples - INFO - Project 'BaldEagleCrkMulti2D' already exists. Deleting existing folder...
2025-12-10 21:10:18 - ras_commander.RasExamples - INFO - Existing folder for project 'BaldEagleCrkMulti2D' has been deleted.
2025-12-10 21:10:19 - ras_commander.RasExamples - INFO - Successfully extracted project 'BaldEagleCrkMulti2D' to c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D
2025-12-10 21:10:19 - ras_commander.RasMap - INFO - Successfully

Project extracted to: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D

Project: BaldEagleDamBrk
Available plans: ['13', '15', '17', '18', '19', '03', '04', '02', '01', '05', '06']


---
# Part 2: Local Parallel Execution

Use `RasCmdr.compute_parallel()` to run multiple plans on your **local machine**.
This is the simplest way to parallelize HEC-RAS execution.

#### Configuration for local parallel execution
LOCAL_PLANS = ["03", "04", "06"]  # Plans to execute
LOCAL_MAX_WORKERS = 3              # Number of parallel processes
LOCAL_NUM_CORES = 2                # Cores per HEC-RAS instance

print(f"Plans to execute: {LOCAL_PLANS}")
print(f"Configuration: {LOCAL_NUM_CORES} cores x {LOCAL_MAX_WORKERS} workers")

#### Execute plans in parallel on LOCAL machine
print(f"Starting LOCAL parallel execution of {len(LOCAL_PLANS)} plans...")
print("="*70)

start_time = time.time()

local_results = RasCmdr.compute_parallel(
    plan_number=LOCAL_PLANS,
    max_workers=LOCAL_MAX_WORKERS,
    num_cores=LOCAL_NUM_CORES,
    ras_object=ras,
    overwrite_dest=True
)

elapsed = time.time() - start_time

print(f"\nExecution complete in {elapsed:.1f} seconds ({elapsed/60:.1f} minutes)")

#### Display local execution results
print("LOCAL Execution Results:")
print("="*70)

success_count = sum(1 for success in local_results.values() if success)
fail_count = len(local_results) - success_count

for plan_num, success in local_results.items():
    status = "SUCCESS" if success else "FAILED"
    print(f"  Plan {plan_num}: {status}")

print(f"\nSummary: {success_count}/{len(local_results)} plans succeeded")
print(f"Results folder: {ras.project_folder.parent / (ras.project_folder.name + ' [Computed]')}")

---
# Part 3: Remote Execution Setup

To use **remote execution**, you need to configure remote workers.

## First Time Setup
1. Copy `RemoteWorkers.json.template` to `RemoteWorkers.json`
2. Edit `RemoteWorkers.json` with your remote machine details
3. The JSON file is in `.gitignore` for security

## Worker Types
- **local** - Execute on local machine (useful for mixed local+remote)
- **psexec** - Execute on remote Windows via PsExec
- **docker** - Execute in Docker container (local or remote via SSH)

## JSON Configuration Example
```json
{
  "workers": [
    {
      "name": "Local Worker",
      "worker_type": "local",
      "worker_folder": "C:\\RasRemote",
      "cores_total": 8,
      "cores_per_plan": 2,
      "enabled": true
    },
    {
      "name": "Remote Workstation",
      "worker_type": "psexec",
      "hostname": "192.168.1.100",
      "share_path": "\\\\192.168.1.100\\RasRemote",
      "worker_folder": "C:\\RasRemote",
      "username": "your_username",
      "password": "your_password",
      "session_id": 2,
      "cores_total": 16,
      "cores_per_plan": 4,
      "enabled": true
    }
  ]
}
```

In [3]:
# Check if RemoteWorkers.json exists
config_file = Path("RemoteWorkers.json")

if not config_file.exists():
    print("WARNING: RemoteWorkers.json not found!")
    print()
    print("To use remote execution:")
    print("1. Copy RemoteWorkers.json.template to RemoteWorkers.json")
    print("2. Edit RemoteWorkers.json with your remote machine details")
    print("3. Re-run this cell")
    print()
    print("For now, you can still use LOCAL parallel execution (Part 2 above).")
    REMOTE_AVAILABLE = False
else:
    import json
    with open(config_file, 'r') as f:
        worker_configs = json.load(f)
    
    enabled_configs = [w for w in worker_configs["workers"] if w.get("enabled", True)]
    
    print(f"Found {len(enabled_configs)} enabled worker(s) in RemoteWorkers.json:")
    for w in enabled_configs:
        cores_total = w.get('cores_total', 'Not set')
        cores_per_plan = w.get('cores_per_plan', 4)
        print(f"  - {w.get('name', 'unnamed')} ({w.get('worker_type', 'unknown')})")
        print(f"    Cores: {cores_total} total, {cores_per_plan} per plan")
    
    REMOTE_AVAILABLE = True

Found 4 enabled worker(s) in RemoteWorkers.json:
  - CLB-04 (psexec)
    Cores: 8 total, 2 per plan
  - Local Compute (local)
    Cores: 8 total, 2 per plan
  - CLB-02 Docker 6.6 (docker)
    Cores: 6 total, 2 per plan
  - CLB-03 Docker 6.6 (docker)
    Cores: 6 total, 2 per plan


---
# Part 4: Remote Parallel Execution

Use `compute_parallel_remote()` to run plans on **remote machines**.

**Key differences from local execution:**
- Plans are copied to remote shares
- Execution happens on remote machines
- Results are collected back to local project
- Supports multiple remote workers simultaneously

In [ ]:
# Load remote workers from JSON
# NOTE: This must be called AFTER init_ras_project() so ras_exe_path is available

if REMOTE_AVAILABLE:
    workers = load_workers_from_json("RemoteWorkers.json")
    
    print(f"Loaded {len(workers)} worker(s):")
    for w in workers:
        print(f"  - {w.worker_id} ({w.worker_type})")
        if hasattr(w, 'max_parallel_plans') and w.max_parallel_plans > 1:
            print(f"    Parallel Capacity: {w.max_parallel_plans} plans")
else:
    print("Remote workers not available - configure RemoteWorkers.json first")
    workers = []

2025-12-10 21:10:19 - ras_commander.remote.RasWorker - INFO - Initializing psexec worker
2025-12-10 21:10:19 - ras_commander.remote.PsexecWorker - INFO - Initializing PsExec worker for 192.168.3.8
2025-12-10 21:10:19 - ras_commander.remote.PsexecWorker - INFO - PsExec worker configured:
2025-12-10 21:10:19 - ras_commander.remote.PsexecWorker - INFO -   Hostname: 192.168.3.8
2025-12-10 21:10:19 - ras_commander.remote.PsexecWorker - INFO -   Share path: \\192.168.3.8\RasRemote
2025-12-10 21:10:19 - ras_commander.remote.PsexecWorker - INFO -   Worker folder: C:\RasRemote
2025-12-10 21:10:19 - ras_commander.remote.PsexecWorker - INFO -   User: .\bill
2025-12-10 21:10:19 - ras_commander.remote.PsexecWorker - INFO -   System account: False
2025-12-10 21:10:19 - ras_commander.remote.PsexecWorker - INFO -   Session ID: 2
2025-12-10 21:10:19 - ras_commander.remote.PsexecWorker - INFO -   Process Priority: low
2025-12-10 21:10:19 - ras_commander.remote.PsexecWorker - INFO -   Queue Priority: 1
2

In [ ]:
# Configuration for remote execution
REMOTE_PLANS = ["03", "04", "06"]  # Plans to execute remotely
REMOTE_NUM_CORES = 4               # Cores per HEC-RAS instance

print(f"Plans to execute: {REMOTE_PLANS}")
print(f"Workers available: {len(workers)}")

In [ ]:
# Execute plans on REMOTE workers
if workers:
    print(f"Starting REMOTE execution of {len(REMOTE_PLANS)} plans...")
    print(f"Using {len(workers)} worker(s)")
    print("="*70)
    
    start_time = time.time()
    
    remote_results = compute_parallel_remote(
        plan_numbers=REMOTE_PLANS,
        workers=workers,
        num_cores=REMOTE_NUM_CORES,
        autoclean=True  # Delete temp folders after execution (default)
    )
    
    elapsed = time.time() - start_time
    
    print(f"\nExecution complete in {elapsed:.1f} seconds ({elapsed/60:.1f} minutes)")
else:
    print("No remote workers available.")
    print("Configure RemoteWorkers.json to enable remote execution.")
    remote_results = {}

In [ ]:
# Display remote execution results
if remote_results:
    print("REMOTE Execution Results:")
    print("="*70)
    
    for plan_num, result in remote_results.items():
        if result.success:
            print(f"  Plan {plan_num}: SUCCESS ({result.execution_time:.1f}s)")
            print(f"    HDF Path: {result.hdf_path}")
        else:
            print(f"  Plan {plan_num}: FAILED - {result.error_message}")
    
    success_count = sum(1 for r in remote_results.values() if r.success)
    print(f"\nSummary: {success_count}/{len(remote_results)} plans succeeded")
else:
    print("No remote results - run the remote execution cell above first.")

---
# Part 5: Verify Results

Check that HDF files were created and contain valid results.

In [ ]:
# Verify HDF results
from ras_commander import HdfResultsPlan

print("Result Verification:")
print("="*70)

# Check plans that were executed (either local or remote)
plans_to_check = LOCAL_PLANS if local_results else (REMOTE_PLANS if remote_results else [])

for plan_num in plans_to_check:
    hdf_path = project_path / f"{ras.project_name}.p{plan_num}.hdf"
    
    if hdf_path.exists():
        size_mb = hdf_path.stat().st_size / (1024 * 1024)
        print(f"\nPlan {plan_num}:")
        print(f"  HDF Size: {size_mb:.2f} MB")
        
        # Get compute messages
        msgs = HdfResultsPlan.get_compute_messages(hdf_path)
        if "completed successfully" in msgs.lower() or "complete process" in msgs.lower():
            print(f"  Status: SUCCESS")
        else:
            print(f"  Status: Check messages")
        
        # Get volume accounting
        vol = HdfResultsPlan.get_volume_accounting(hdf_path)
        if vol is not None and len(vol) > 0:
            error_pct = vol['Error Percent'].iloc[0]
            print(f"  Volume Error: {error_pct:.4f}%")
    else:
        print(f"\nPlan {plan_num}: HDF not found")

---
# Part 6: Manual Worker Configuration (Optional)

You can also create workers programmatically without a JSON file.

In [ ]:
# Example: Create a PsExec worker manually
# Uncomment and modify with your settings to test

# manual_worker = init_ras_worker(
#     "psexec",
#     hostname="192.168.1.100",
#     share_path=r"\\192.168.1.100\RasRemote",
#     worker_folder=r"C:\RasRemote",
#     credentials={
#         "username": "your_username",
#         "password": "your_password"
#     },
#     session_id=2,
#     cores_total=8,
#     cores_per_plan=2
# )
# 
# print(f"Manual worker created: {manual_worker.worker_id}")
# print(f"  Parallel Capacity: {manual_worker.max_parallel_plans} plans")

print("Uncomment the code above to create a manual worker.")

---
# Part 7: Cleanup (Optional)

Clean up temporary worker folders on remote shares.

In [ ]:
# Cleanup function for remote shares
def cleanup_remote_shares(workers, dry_run=True):
    """Clean up worker folders from remote shares."""
    import shutil
    
    seen_shares = set()
    
    for w in workers:
        if not hasattr(w, 'share_path') or not w.share_path:
            continue
            
        share_path = Path(w.share_path)
        if str(share_path) in seen_shares:
            continue
        seen_shares.add(str(share_path))
        
        # Check if share exists before accessing
        if not share_path.exists():
            continue
                
            folders = [f for f in share_path.iterdir() if f.is_dir()]
            
            print(f"\nShare: {share_path}")
            print(f"  Folders: {len(folders)}")
            
            for folder in folders:
                folder_size = sum(f.stat().st_size for f in folder.rglob('*') if f.is_file())
                folder_size_mb = folder_size / (1024 * 1024)
                
                if dry_run:
                    print(f"  [WOULD DELETE] {folder.name} ({folder_size_mb:.1f} MB)")
                else:
                    print(f"  [DELETING] {folder.name}")
                    shutil.rmtree(folder, ignore_errors=True)
                    
        except Exception as e:
            print(f"Error: {e}")

# Preview cleanup (dry run)
if workers:
    print("CLEANUP PREVIEW (dry_run=True):")
    cleanup_remote_shares(workers, dry_run=True)
    print("\nSet dry_run=False to actually delete folders.")
else:
    print("No workers loaded - nothing to clean up.")

---
# Summary: Local vs Remote Execution

| Feature | Local (`compute_parallel`) | Remote (`compute_parallel_remote`) |
|---------|---------------------------|------------------------------------|
| Setup | None | Configure RemoteWorkers.json |
| Execution | Local machine only | Remote machines via PsExec/Docker |
| File Transfer | Direct | Via network shares |
| Scaling | Limited by local cores | Unlimited remote machines |
| Best For | Small jobs, testing | Large batches, distributed work |

## Quick Reference

**Local Parallel:**
```python
results = RasCmdr.compute_parallel(
    plan_number=["01", "02", "03"],
    max_workers=3,
    num_cores=2,
    ras_object=ras
)
```

**Remote Parallel:**
```python
workers = load_workers_from_json("RemoteWorkers.json")
results = compute_parallel_remote(
    plan_numbers=["01", "02", "03"],
    workers=workers,
    num_cores=4
)
```

---

**For complete setup instructions, see:**
- `feature_dev_notes/RasRemote/REMOTE_WORKER_SETUP_GUIDE.md`